#### Dependencies

In [1]:
from dataset_reader import Traces_Dataset
from DE_obj_model import de_obj_model   
from exp_hh_model import HH_model_exp
import numpy as np
import matplotlib.pyplot as plt

#### Read test dataset

In [2]:
dataset = Traces_Dataset('dataset_test.csv')

sample = 1
params = dataset.params.numpy()
current_traces = dataset.current_traces.numpy()
time_traces = dataset.time_traces.numpy()

prestep_V_2d = dataset.prestep_V[sample].numpy().reshape(-1,1)
step_Vs_2d = dataset.step_Vs[sample].numpy().reshape(-1,1)
t = time_traces[sample]
# target_traces = current_traces[sample]
target_params = params[sample]

In [3]:
# sim setup for obj evaluation model
sim_setup_2d = {'prestep_V': prestep_V_2d, 'step_Vs': step_Vs_2d, 't': t}

# model = de_obj_model(target_params, sim_setup_2d)

In [27]:
# sim_traces = model.simulation()
# sim_traces.shape

#### Plotting function for trail, target, and real data

In [6]:
def plot_trail_target_real(sim_setup_1d, t, sim_traces, target_traces, target_params): 
    '''
    sim_setup_1d is the exp setup for hh model, all default as in hh class
    t is the target time traces from generated dataset
    sim_traces is the generated during searching stage
    target_traces is the target current traces from the generated dataset
    target_params is the target params from the generated dataset, dict
    '''
    # compare samples and simulations using sample params
    hh_model = HH_model_exp(target_params, sim_setup_1d)
    current_traces_sim = hh_model.simulation()

    colors = ['blue', 'red', 'green', 'purple', 'orange', 'yellow', 'cyan', 'magenta', 'brown', 'gray', 'black']

    for step in range(t.shape[0]): 
        plt.plot(t[step], sim_traces[step], linestyle='-', color=colors[step])
        plt.plot(t[step], target_traces[step], linestyle=':', color=colors[step])
        plt.plot(sim_setup_1d['t'], current_traces_sim[step], linestyle='--', color=colors[step])
    plt.legend()
    plt.title(f"prestep_V: {sim_setup_1d['prestep_V']}; step_V1: {sim_setup_1d['step_Vs'][0]}")


In [7]:
# sim setup for plotting real data
sim_setup_1d = {'prestep_V': np.squeeze(prestep_V_2d), 'step_Vs': np.squeeze(step_Vs_2d), 't': np.arange(0.0, 6.0, 0.01)}
target_params_dict = {'p': target_params[0], 'g_max': target_params[1], 'E_rev': target_params[2], 'a_m': target_params[3], 'b_m': target_params[4], 'delta_m': target_params[5], 's_m': target_params[6]}
plot_trail_target_real(sim_setup_1d, t, sim_traces, target_traces, target_params_dict)

NameError: name 'sim_traces' is not defined

#### Define obj function

In [4]:
def obj(x, *args): 
    '''
    x: a 1-D array of the variables for the obj function (the parameters we are estimating)
    args: a tupleo f additional fixed parameters (prestep_V, step_V0, time_traces)
    *args=(sim_setup_2d, target_params)
    '''
    trail_model = de_obj_model(x, args[0])
    trail_traces = trail_model.simulation()
    # print(trail_traces[1])
    target_model = de_obj_model(args[1], args[0])
    target_traces = target_model.simulation()
    # print(target_traces[1]) 
    
    return np.sum(np.square(trail_traces - target_traces))


In [5]:
fitness = obj(params[sample], *(sim_setup_2d, target_params))
print(fitness)

0.0


#### Define bounds

In [6]:
# these bounds are from the distribution of the params in the dataset used for NN training
params_searching_bounds = {
    'p': (1, 4),
    'g_max': (100, 140), 
    'E_rev': (-100, -60), 
    'a_m': (0, 100), 
    'b_m': (0, 100), 
    'delta_m': (0, 1), 
    's_m': (-100, 0)
}
bounds = [params_searching_bounds['p'], params_searching_bounds['g_max'], params_searching_bounds['E_rev'], params_searching_bounds['a_m'], params_searching_bounds['b_m'], params_searching_bounds['delta_m'], params_searching_bounds['s_m']]

In [14]:
# def callback_function(xk, convergence):
#     callback_function.iteration += 1
#     print("Iteration:", callback_function.iteration)
#     print("Solution:", xk)
#     print("Objective value:", obj(xk, sim_setup_2d, target_params))
#     print("------------------------")
# callback_function.iteration = 0  # Initialize the iteration counter

In [16]:
from scipy.optimize import differential_evolution
result = differential_evolution(obj, bounds, args=(sim_setup_2d, target_params), maxiter=1000)#, callback=callback_function)

/Users/maxwellyue/Documents/GitHub/Compute-Canada-Research/Exp_rates_generalized_exp_setup/DE_obj_model.py:42: RuntimeWarning: invalid value encountered in divide
  return ((tau_0m * np.exp(self.delta_m * ((V - self.V_2m) / self.s_m))) / (1 + np.exp((V - self.V_2m) / self.s_m))).reshape((-1,1))


In [8]:
target_params, result.x

(array([  1.        , 133.65648   , -81.39668   ,   4.1990376 ,
         12.4988    ,   0.39262012, -11.015537  ], dtype=float32),
 array([  1.        , 133.65648479, -81.3966775 ,   4.1990373 ,
         12.49880051,   0.39262009, -11.01553655]))

In [9]:
((target_params - result.x) ** 2)

array([5.02251217e-22, 3.48942953e-11, 2.74724105e-11, 6.20176264e-14,
       5.45464608e-14, 4.73896655e-16, 5.02978803e-13])

In [10]:
result.x

array([  1.        , 133.65648479, -81.3966775 ,   4.1990373 ,
        12.49880051,   0.39262009, -11.01553655])

In [35]:
result

 message: Optimization terminated successfully.
 success: True
     fun: 6.48722942770685e-06
       x: [ 1.000e+00  1.337e+02 -8.140e+01  4.199e+00  1.250e+01
            3.926e-01 -1.102e+01]
     nit: 772
    nfev: 81197
     jac: [ 4.200e+01  6.163e-03  4.059e-03  0.000e+00  0.000e+00
            0.000e+00  0.000e+00]

#### Run DE on all test dataset, check result mse on all params here

In [16]:
mse_mat = np.load('de_mse_params_all_test.npy')

In [17]:
mse_mat.shape

(30, 7)

In [18]:
mse_params = np.mean(mse_mat, axis=0)
mse_params

array([1.68353114e-03, 5.70682053e+01, 1.12901201e+02, 2.79381230e-03,
       7.58826951e+00, 1.83145674e-04, 7.30552287e-02])

## Define obj fct and estimate for single param

In [20]:
def obj_i_param(x, *args): 
    '''
    x: a single float value of the i-th param
    args: a tupleo f additional fixed parameters (prestep_V, step_V0, time_traces)
    *args=(i, sim_setup_2d, target_params)
    '''
    params = args[2].copy()
    params[args[0]] = x
    trail_model = de_obj_model(params, args[1])
    trail_traces = trail_model.simulation()
    # print(trail_traces[1])
    target_model = de_obj_model(args[2], args[1])
    target_traces = target_model.simulation()
    # print(target_traces[1]) 
    
    return np.sum(np.square(trail_traces - target_traces))

In [21]:
params_searching_bounds = {
    'p': (1, 5),
    'g_max': (100, 140), 
    'E_rev': (-100, -60), 
    'a_m': (0, 13), 
    'b_m': (0, 100), 
    'delta_m': (0, 1), 
    's_m': (-17, -10)
}
i = 6
key_at_index = list(params_searching_bounds.keys())[i]
bounds = [params_searching_bounds[key_at_index]]
bounds

[(-17, -10)]

In [22]:
from scipy.optimize import differential_evolution
result = differential_evolution(obj_i_param, bounds, args=(i, sim_setup_2d, target_params), maxiter=100)#, tol = 1e-10)
result

 message: Optimization terminated successfully.
 success: True
     fun: 0.0
       x: [-1.105e+01]
     nit: 22
    nfev: 347

In [32]:
((target_params[i] - result.x) ** 2)

array([2.0789837e-15])

#### Results for all single param estimation

In [27]:
mses = np.load('de_mses_on_ith_params.npy')
nits = np.load('de_nit_on_ith_params.npy')
mses.shape, nits.shape

((30, 7), (30, 7))

In [28]:
arr = mses.mean(axis=0)
print(arr)

sorted_indices = np.argsort(arr)

print("Sorted indices:", sorted_indices)

[1.40752475e-15 6.77700377e-15 2.20245039e-15 1.40308669e-15
 2.14625476e-15 1.44141350e-15 2.20503233e-15]
Sorted indices: [3 0 5 4 2 6 1]


In [29]:
arr = nits.mean(axis=0)
print(arr)

sorted_indices = np.argsort(arr)

print("Sorted indices:", sorted_indices)

[27.         26.1        26.53333333 26.53333333 26.83333333 27.23333333
 26.33333333]
Sorted indices: [1 6 2 3 4 0 5]
